In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

# Set the path to your Google Cloud Vision API JSON key file
credentials_path = '/home/goutam/Downloads/clever-courier-412909-a58ab09d7051.json'


class TextExtractorApp:
    def __init__(self, root):
        self.root = root
        self.root.geometry("500x600")
        self.root.title("Text Extractor App")

        # Create widgets
        self.label = tk.Label(root, text="Choose an image:", font=('Arial', 15))
        self.label.pack(padx=100, pady=10)

        self.choose_button = tk.Button(self.root, text="Choose Image", font=('Arial', 15), command=self.choose_image)
        self.choose_button.pack(pady=20)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.result_label = tk.Label(root, text="Processed Text:", font=('Arial', 12))
        self.result_label.pack(pady=10)

        self.result_text = tk.Text(root, height=15, width=50)
        self.result_text.pack(pady=10)

        # Initialize Google Cloud Vision API client
        try:
            self.client = vision_v1.ImageAnnotatorClient.from_service_account_file(credentials_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to initialize Google Cloud Vision client: {e}")
            self.client = None

    def choose_image(self):
        if not self.client:
            messagebox.showerror("Error", "Google Cloud Vision client not initialized.")
            return

        try:
            file_path = filedialog.askopenfilename(
                initialdir="/",
                title="Select an image file",
                filetypes=[("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")]
            )

            if file_path:
                self.display_image(file_path)
                text = self.extract_text(file_path)
                self.result_text.delete(1.0, tk.END)
                self.result_text.insert(tk.END, text)
            else:
                messagebox.showinfo("Info", "No image selected.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def display_image(self, file_path):
        image = Image.open(file_path)
        image = image.resize((250, 250))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def extract_text(self, file_path):
        try:
            with open(file_path, 'rb') as image_file:
                content = image_file.read()

            image = types.Image(content=content)
            response = self.client.text_detection(image=image)
            texts = response.text_annotations

            if texts:
                return texts[0].description.strip()
            else:
                return "No text found."
        except Exception as e:
            return f"An error occurred during text extraction: {e}"


if __name__ == "__main__":
    root = tk.Tk()
    app = TextExtractorApp(root)
    root.mainloop()
